In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("transformations_actions")

In [3]:
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/07 00:31:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/07 00:31:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '51402'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1636081026534'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '172.30.1.34'),
 ('spark.app.id', 'local-1636081027829'),
 ('spark.ui.showConsoleProgress', 'true')]

In [4]:
# SparkContext는 하나만 생성할 수 있고 새로 생성하고 싶을 경우 기존 sc를 종료해야한다.
sc.stop()

In [5]:
sc = SparkContext(conf=conf)

22/01/07 00:32:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
# parallelize: list로부터 데이터를 만듦
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면"])

In [10]:
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [13]:
# RDD안 모든 데이터를 확인하고 싶을 경우 사용
# 실제 product에서 사용하면 안되는 코드
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [12]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [14]:
# n개의 데이터만 뽑아보고 싶을 떄 사용
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [15]:
foods.take(5)

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수']

In [16]:
# 첫번째 데이터 확인
foods.first()

'짜장면'

In [17]:
# count: action
foods.count()

12

In [19]:
# distinct: transformation
foods.distinct().collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [18]:
# distinct: transformation
foods.distinct().count()

7

In [20]:
# foreach: element를 하나씩 꺼내서 하나의 함수를 적용, action
# worker node 어딘가에서는 실행이 되나 jupyter notebook은 driver program이기에 결과물이 보이지 않음
# RDD를 연산하고 log를 저장해 두는데 유용
foods.foreach(lambda x: print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


## narrow transformation

In [22]:
sc.parallelize([1, 2, 3]).map(lambda x: x + 2).collect()

[3, 4, 5]

In [23]:
sc.parallelize([1, 2, 3]).map(lambda x: x * 2).collect()

[2, 4, 6]

In [24]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [25]:
moviesRDD = sc.parallelize(movies)

In [26]:
moviesRDD.collect()

['그린 북',
 '매트릭스',
 '토이 스토리',
 '캐스트 어웨이',
 '포드 V 페라리',
 '보헤미안 랩소디',
 '빽 투 더 퓨처',
 '반지의 제왕',
 '죽은 시인의 사회']

In [27]:
flatMovies = moviesRDD.flatMap(lambda x: x.split(" "))

In [28]:
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [31]:
filteredMovies = flatMovies.filter(lambda x: x != "매트릭스")

In [32]:
filteredMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [33]:
num1 = sc.parallelize([1, 2, 3, 4])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [34]:
num1.intersection(num2).collect()

[4]

In [35]:
num1.union(num2).collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [36]:
num1.subtract(num2).collect()

[2, 1, 3]

In [37]:
numUnion = num1.union(num2)

In [38]:
numUnion.collect()

[1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]

In [39]:
# numUnion.sample(withReplacement, proportion, seed)
# [1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10]
numUnion.sample(True, .5).collect()


[2, 4, 4, 4, 5, 10]

In [40]:
numUnion.sample(True, .5, seed=5).collect()


[1, 4, 6, 9, 9, 10]

## wide transformation

In [41]:
foods = sc.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면",  "라면", "우동", "라면", "치킨", "돈까스", "회", "햄버거", "피자"])

In [43]:
# groupBy
foodsGroup = foods.groupBy(lambda x: x[0])

In [44]:
res = foodsGroup.collect()

In [45]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
치 ['치킨']
돈 ['돈까스']
회 ['회']
햄 ['햄버거']
피 ['피자']


In [46]:
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

In [49]:
list(nums.groupBy(lambda x: x % 2).collect())

[(1, <pyspark.resultiterable.ResultIterable at 0x7f87cf03dd90>),
 (0, <pyspark.resultiterable.ResultIterable at 0x7f87cf065a00>)]

In [47]:
list(nums.groupBy(lambda x: x % 2).collect()[1][1])

[2, 4, 6, 8, 10]